### Implementing and Logging an ML Experiment with MLFlow
**Description**: Train an ML model for an e-commerce recommendation engine using MLFlow to track models and experiments.

**Steps**:
1. MLFlow Integration Setup
2. Training the Model
3. Logging the Experiment
4. Accessing MLFlow UI

In [ ]:
# write your code from here

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, classification_report
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler
import os

# -----------------------------
# STEP 1: Simulate E-commerce Data
# -----------------------------
def create_synthetic_data():
    np.random.seed(42)
    n = 1000
    data = pd.DataFrame({
        "user_age": np.random.randint(18, 65, n),
        "product_views": np.random.poisson(3, n),
        "cart_adds": np.random.poisson(1, n),
        "purchase_made": np.random.choice([0, 1], size=n, p=[0.7, 0.3])
    })
    return data

# -----------------------------
# STEP 2: Train ML Model
# -----------------------------
def train_model(data):
    X = data.drop("purchase_made", axis=1)
    y = data["purchase_made"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = LogisticRegression()
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)

    print("[INFO] Model Training Completed.")
    print("[INFO] Accuracy:", acc)
    print("[INFO] Precision:", prec)
    
    return model, acc, prec, scaler

# -----------------------------
# STEP 3: MLflow Logging
# -----------------------------
def log_experiment(model, accuracy, precision, scaler):
    mlflow.set_experiment("E-Commerce-Recommendation-Engine")

    with mlflow.start_run():
        mlflow.log_param("model_type", "LogisticRegression")
        mlflow.log_param("scaler", "StandardScaler")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)

        # Log model
        mlflow.sklearn.log_model(model, "model")
        mlflow.sklearn.log_model(scaler, "scaler")
        print("[MLflow] Experiment logged successfully.")

# -----------------------------
# MAIN: Full Pipeline Execution
# -----------------------------
if __name__ == "__main__":
    print("=== MLflow Experiment: E-Commerce Recommendation ===\n")

    # Step 1: Generate data
    df = create_synthetic_data()

    # Step 2: Train model
    model, acc, prec, scaler = train_model(df)

    # Step 3: Log to MLflow
    log_experiment(model, acc, prec, scaler)

    print("\n=== DONE: Check the MLflow UI with `mlflow ui` ===")